In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from urllib.parse import quote_plus
import re
import csv

https://api.stlouisfed.org/fred/category/series?category_id=125&api_key=abcdefghijklmnopqrstuvwxyz123456
https://fred.stlouisfed.org/docs/api/fred/category_series.html#Description 

In [2]:
# Set environment variables from the .env in the local environment    

def key_check(key_path=None):
    try:
        reply=load_dotenv(key_path,verbose=True,override=True)   
        assert reply , 'Dotenv is not found'
        fred_api_key = os.getenv("FRED_API_KEY")
        display (fred_api_key)
        assert fred_api_key is not None, 'FRED_API_KEY not found in .env file'
        responce=requests.get(f'https://api.stlouisfed.org/fred/category/series?category_id=125&api_key={fred_api_key}')
        assert responce.status_code == 200, f'The key provided failed to authenticate tmdb_api_key {fred_api_key} code {responce.status_code}'
    except Exception as e:
        # Handle potential errors in loading .env or missing API keys
        print(f'An error occurred: {e}')
        return(False)
    else:
        print('All keys loaded correctly')
        return (True)
my_env_path='C:\SRC\AI\ProjectOne_Team2\Frank_playground\keys.env'
if key_check(my_env_path):
    fred_api_key = os.getenv("FRED_API_KEY")
else:
    print ('fix Keys and rerun')

'791aff06728f5b2b7056c7e789b48ad5'

All keys loaded correctly


In [3]:
# this cell will be used to read the Series ID from the series ID file
# read the file into a list
list_of_series_df = pd.read_csv('C:\SRC\AI\ProjectOne_Team2\Frank_playground\list_of_series.csv')
# client_data_df.head()
display (list_of_series_df.head(3))

# build a Filename to save the File
# Set the series_id

,country,state,county,series_Desc,series_id
0,United States,TX,Collin,House Price Index,ATNHPIUS48085A
1,United States,TX,Dallas,House Price Index,ATNHPIUS48113A
2,United States,TX,Denton,House Price Index,ATNHPIUS48121A


In [4]:
#  Set the base URL
series_id='ATNHPIUS48113A'

url = (f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={fred_api_key}&file_type=json")
display (url)

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL


'https://api.stlouisfed.org/fred/series/observations?series_id=ATNHPIUS48113A&api_key=791aff06728f5b2b7056c7e789b48ad5&file_type=json'

In [7]:
def create_filename(row):
    # Join the desired fields with underscores
    filename = '_'.join(row[['country', 'state', 'county', 'series_Desc']])
    # Replace spaces with underscores and remove any problematic characters
    filename = filename.replace(' ', '_').replace('/', '_')
    return filename

for index, row in list_of_series_df.iterrows():
    display ({index})
    series_id=list_of_series_df.loc[index,'series_id']
    display (series_id)
    url = (f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={fred_api_key}&file_type=json")
    display (url)
    series = requests.get(url).json()
    # Create an empty list to store the reviews
    series_list =[]
    for item in series["observations"]:
        series_list.append(item)
        
    series_df=pd.DataFrame(series_list)
    series_df=series_df.drop(series_df.columns[[0,1]],axis=1)
    series_df['date']=pd.to_datetime(series_df['date'])
    series_df.set_index('date',inplace=True)
    display (series_df.head(3))
    
    filename = create_filename(row)
    display (filename)
    file_path = f"./resource/{filename}.csv"  # Construct file path with .csv extension

    series_df.to_csv(file_path, index=True)

{0}

'ATNHPIUS48085A'

'https://api.stlouisfed.org/fred/series/observations?series_id=ATNHPIUS48085A&api_key=791aff06728f5b2b7056c7e789b48ad5&file_type=json'

,value
date,
1975-01-01,36.23
1976-01-01,40.70
1977-01-01,43.91


'United_States_TX_Collin_House_Price_Index'

...

{8}

'TXDENT2POP'

'https://api.stlouisfed.org/fred/series/observations?series_id=TXDENT2POP&api_key=791aff06728f5b2b7056c7e789b48ad5&file_type=json'

,value
date,
1970-01-01,75.633
1971-01-01,81.300
1972-01-01,89.500


'United_States_TX_Denton_Resident_Population'